In [1]:
# 아래처럼 설정
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.38.0 peft==0.4.0 accelerate==0.21.0 trl==0.7.11
%pip install -q -U datasets py7zr einops tensorboardX

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] = cuda_install_dir

In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-04-24 05:55:48.265697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import os
# 허깅페이스 모델/토크나이저를 다운로드 받을 경로
# (예시)
# os.environ['HF_HOME'] = '/home/jovyan/work/tmp'
os.environ['HF_HOME'] = './model'

In [6]:
BASE_MODEL = "google/gemma-1.1-7b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like google/gemma-1.1-7b-it is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
query = '''
경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
경사의 경우, 본인 결혼 시 5일의 휴가와 50만 원의 금액, 화환이 지급됩니다.
자녀 결혼 시 1일의 휴가와 30만 원의 금액, 화환이 지급됩니다.
형제자매 결혼 시 1일의 휴가와 10만 원의 금액이 지급됩니다.
자녀 출산(배우자) 시 10일의 휴가와 10만 원의 금액이 지급되며, 1회 분할 사용이 가능합니다.
부모 회갑 시 1일의 휴가와 20만 원의 금액이 지급됩니다.
배우자 부모 회갑 시 1일의 휴가와 20만 원의 금액이 지급됩니다.
부모 고희(칠순) 시 1일의 휴가와 30만 원의 금액이 지급됩니다.
배우자 부모 고희 시 1일의 휴가와 30만 원의 금액이 지급됩니다.
조사의 경우, 본인 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
배우자 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
부모 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
자녀 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
배우자 부모 사망 시 5일의 휴가와 50만 원의 금액, 조화가 지급됩니다.
형제자매 사망 시 3일의 휴가와 30만 원의 금액, 조화가 지급됩니다.
조부모/외조부모 사망 시 3일의 휴가와 20만 원의 금액이 지급됩니다.

1. 목적
이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.
2. 적용범위
직원의 급여는 관계법령, 기타 별도로 정한 것을 제외하고는 이 규정이 정하는 바에 따른다.
3. 급여의 구성
3.1 급여는 기본급, 제수당, 상여금, 퇴직금으로 구분한다.
3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.
4. 급여조정
급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.

Q: 부모가 돌아가셨을때 유급휴가는 며칠?
'''

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
messages = [
    {
        "role": "user",
        "content": "{}".format(query)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(prompt)

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])